# <center> Sauvegarder et recharger un modèle de Deep Learning avec Keras </center>

**Source :** Cours de Franck Bardol, [LinkedIn Learning](https://www.linkedin.com/learning/decouvrir-le-deep-learning-avec-keras/bienvenue-dans-le-deep-learning-avec-keras?autoplay=true).

## Objectif

Nous allons voir ensemble comment 
* sauvegarder un modèle sur un fichier
* recharger un modèle à partir d'un fichier

## Data set
Données médicales : classification de cellules cancéreuses 

## Avertissement
Pour executer le code sans erreur, vous **devez avoir un compte Google**

In [ ]:
#
#               CANCER DATA
#
from sklearn import datasets
from sklearn import model_selection

import numpy as np
from keras.models import Sequential
from keras.layers import Dense

from keras.models import model_from_json

# fix random seed 
seed = 12345
np.random.seed(seed)

data = datasets.load_breast_cancer()
print("classe = {}".format(np.unique(data.target)))
print("features :")
print(data.feature_names)

# taille du data-set (les features : les colonnes)
n_feature = data.feature_names.shape[0]
print("taille du data set = " , n_feature)

Using TensorFlow backend.


classe = [0 1]
features :
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
('taille du data set = ', 30)


In [ ]:
X = data.data
y = data.target

### data set 
On voit que c'est un problème  classification **binaire** (vrai / faux) 

En effet : classe = [0 1]


Les données sont des mesures physiques de cellules (périmètre, texture, diamètre, etc ...)


## Le modèle de Deep Learning

In [ ]:
# create-compile-fit model
model = Sequential()
model.add(Dense(n_feature, input_dim = n_feature, activation='sigmoid'))
model.add(Dense(n_feature, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_2 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 1,891
Trainable params: 1,891
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(X , y)

In [ ]:
model.fit(X_train, 
                    y_train, 
                    validation_split = 0.33, 
                    epochs=150, 
                    batch_size=10, 
                    verbose=1)

Train on 285 samples, validate on 141 samples
Epoch 1/150
285/285 [==============================] - 0s 180us/step - loss: 0.2256 - accuracy: 0.9158 - val_loss: 0.2496 - val_accuracy: 0.9291
Epoch 2/150
285/285 [==============================] - 0s 163us/step - loss: 0.1714 - accuracy: 0.9439 - val_loss: 0.2610 - val_accuracy: 0.9078
Epoch 3/150
285/285 [==============================] - 0s 172us/step - loss: 0.1632 - accuracy: 0.9509 - val_loss: 0.2451 - val_accuracy: 0.9149
Epoch 4/150
285/285 [==============================] - 0s 165us/step - loss: 0.1727 - accuracy: 0.9474 - val_loss: 0.2408 - val_accuracy: 0.9220
Epoch 5/150
285/285 [==============================] - 0s 159us/step - loss: 0.1608 - accuracy: 0.9509 - val_loss: 0.2573 - val_accuracy: 0.9149
Epoch 6/150
285/285 [==============================] - 0s 169us/step - loss: 0.1707 - accuracy: 0.9509 - val_loss: 0.2637 - val_accuracy: 0.9078
Epoch 7/150
285/285 [==============================] - 0s 166us/step - loss: 0.1962 

In [ ]:
# evaluate 
scores = model.evaluate(X_test, y_test, verbose=0)
print "% accuracy {:.0f}".format(scores[1]*100)

% accuracy 92


On obtient des performances assez élevées.
Le % de bonnes prédictions sur les données non vues est d'environ 94%

C'est largement assez satisfaisant pour un 1er essai.

On décide de sauvegarder ce modèle sur disque pour l'améliorer plus tard ;-)

##  Google drive en local

On "monte" un disque virtuel ('*gdrive*') sur l'espace de travail ('*content*') alloué par Colab
Ce disque virtuel est situé dans le cloud 

!ls : nous donne la liste des fichiers et des repertoires existants sur notre espace de travail alloué par **Colab**

In [ ]:
!ls

sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!ls

gdrive	sample_data


Si l'opération a réussie.
Vous devez voir un nouveau répertoire nommé **gdrive**

In [ ]:
!ls

gdrive	sample_data


## Sauvegarde de la structure de l'algorithme

on sauve la **structure** de l'algorithme dans un fichier au format approprié (json)

In [ ]:
#====================#====================#====================
# ===================== SAVE  ===================================
#====================#====================#====================

#  model to JSON
print("saving model....")
model_json = model.to_json()

with open('/content/gdrive/My Drive/model.json', 'w') as json_f:
    json_f.write(model_json)

saving model....


## Sauvegarde des paramètres dans un fichier

à présent on sauve les **poids** dans un fichier au format h5 (format binaire compressé)

In [ ]:
#====================#====================#====================
# ===================== SAVE  ===================================
#====================#====================#====================# weights to HDF5
print("saving weights .....")
model.save_weights("/content/gdrive/My Drive/model.h5")

saving weights .....


In [ ]:
print model_json

{"class_name": "Sequential", "keras_version": "2.3.1", "config": {"layers": [{"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_1", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "dtype": "float32", "activation": "sigmoid", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 30, "batch_input_shape": [null, 30], "use_bias": true, "activity_regularizer": null}}, {"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_2", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "dtype": "float32", "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Z

## Lecture du fichier des paramètres 

In [ ]:
#====================#====================
#==================== LOAD ================
#====================#====================
# re-create model
json_f = open( '/content/gdrive/My Drive/model.json' , 'r' )
model_json = json_f.read()
json_f.close()

print("MODEL from JSON: ")
print(json_f)

MODEL from JSON: 
<closed file '/content/gdrive/My Drive/model.json', mode 'r' at 0x7fe11b89fe40>


In [ ]:
load_model = model_from_json(model_json)
# feed loaded model with weights
load_model.load_weights('/content/gdrive/My Drive/model.h5')


## Reconstitution du modèle
Nous avons à présent tous les éléments pour reconstruire le modèle préalablement entrainé

In [ ]:
# compile
load_model.compile(loss= 'binary_crossentropy' , optimizer = 'adam' , metrics=[ 'accuracy' ])

In [ ]:
load_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_2 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 1,891
Trainable params: 1,891
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# evaluate 
scores = model.evaluate(X_test, y_test, verbose=0)
print "% accuracy {:.0f}".format(scores[1]*100)

% accuracy 92


## Conclusion
La performance est identique.
Le modèle chargé depuis le disque est tout à fait identique au modèle initial

## Résumé
Nous savons maintenant 
* comment monter un disque virtuel dans notre environnement cloud
* comment sauvegarder un modèle qui nous donne satisfaction
* comment sauvegarder la structure du modèle (json)
* comment sauvegarder les paramètres (h5)
* comment recharger un modèle stocké sur disque
